#  Madagascar Vulnerability Index - Indicator Explorer Notebook

*author@SingerG, Abt Associates, November 2023*

### Introduction
From the top menu, open the 'Kernel" dropdown and select 'Restart and Run All' in order to start the tool.

The purpose of this tool is enable the user to adjust the weighted domains in the Vulnerability Index.

*If you would like to view each of the indicators and domains in a WebMap, please open the following link in a new window: (LINK TO WEBMAP).*

In [1]:
%%time

def Run_Vulnerability_Index_Explorer(itemID):
    '''
    This is the main function that loads the spatial dataframe from a given item ID. 
    '''
    # Import Modules
    import arcgis
    from arcgis.gis import GIS
    from arcgis.features import FeatureLayer
    from arcgis.features import GeoAccessor, GeoSeriesAccessor
    import pandas as pd
    import ipywidgets as widgets
    from IPython.display import display, HTML
    import io
    import base64
    gis = GIS("home")
    # from arcgis.mapping import display_colormaps
    # display_colormaps()
    # Define function to load Spatially Enabled Dataframe
    def load_sdf(itemid):
        item = gis.content.get(itemid)
        flayer = item.layers[0]
        sdf = pd.DataFrame.spatial.from_layer(flayer)
        return(sdf)

    # Define function to calculate Vulnerability Index (Average of each (Percentile * Weight) per Indicator) 
    def calculate_products_and_average(percentile_rank_df):
        # Create a new DataFrame for the result
        result_df = pd.DataFrame()

        # Iterate through the column sets and calculate the product
        percentile_rank_columns = [col for col in percentile_rank_df.columns if col.endswith('_Percentile_Rank')]
        for col in percentile_rank_columns:
            stem = col.rsplit('_Percentile_Rank', 1)[0]
            weight_col = stem + '_weight'
            result_df[stem] = percentile_rank_df[col] * percentile_rank_df[weight_col]

        # Calculate the row-wise sum
        result_df['Vulnerability_Index'] = round(result_df.sum(axis=1),3)

        # Concatenate the result_df with the original df
        combined_df = pd.concat([percentile_rank_df, result_df], axis=1)
        combined_df['Vulnerability_Index_Percentile'] = round(combined_df['Vulnerability_Index'].rank(pct=True)*100)
        return combined_df

    # Define a function to calculate Percentile_Rank 
    def create_percentile_rank_index(sdf, columns_to_rank):
        # Initialize a new DataFrame to store percentile rank values
        percentile_rank_df = pd.DataFrame()
        # Create an empty list to store the new percentile_rank cols
        percentile_rank_cols = []
        weight_cols = []
        # Calculate percentile ranks for the specified columns and store them in the new DataFrame
        for column in columns_to_rank:
            percentile_rank_df[column + '_Percentile_Rank'] = sdf[column].rank(pct=True)*100
    #         percentile_rank_df[column + '_Percentile_Rank'] = dataframe[column]
            percentile_rank_cols.append(column + '_Percentile_Rank')
            # Add Weight Col
            percentile_rank_df[column + '_weight'] = .1
            weight_cols.append(column + '_weight')       

    #     Add the final index to the new DataFrame
        percentile_rank_df = calculate_products_and_average(percentile_rank_df)
    #     percentile_rank_df['Index'] = (percentile_rank_df[percentile_rank_cols].prod(axis=1)) * (percentile_rank_df[weight_cols].prod(axis=1))
        percentile_rank_df= pd.concat([sdf, percentile_rank_df], axis = 1)
        return percentile_rank_df, percentile_rank_cols, weight_cols

    # Define a function to update the DataFrame with the values from the widgets
    # def update_dataframe(change, output_widget, adj_weight_cols, input_widgets):
    def update_dataframe(change):
        output_widget.clear_output()
        for column in adj_weight_cols:
            df[column] = [input_widgets[column].value] * len(df)

        # Create a new DataFrame for the result
        result_df = pd.DataFrame()

        # Iterate through the column sets and calculate the product
        percentile_rank_columns = [col for col in df.columns if col.endswith('_Percentile_Rank')]
        for col in percentile_rank_columns:
            stem = col.rsplit('_Percentile_Rank', 1)[0]
            weight_col = stem + '_weight'
            result_df[stem] = df[col] * df[weight_col]

        # Calculate the row-wise sum
        df['Vulnerability_Index'] = round(result_df.sum(axis=1),3)
        df['Vulnerability_Index_Percentile'] = round(df['Vulnerability_Index'].rank(pct=True)*100)
    #     calculate_products_and_average(df)
        with output_widget:
            display(df[['OBJECTID', 'SOURCE_ID', 'Vulnerability_Index','Vulnerability_Index_Percentile']])

    # Define a function to update the map 
    def update_map(change):
        # Remove original layer
        m1.remove_layers()    

        df[['SHAPE','Vulnerability_Index_Percentile']].spatial.plot(map_widget = m1,
                                                             renderer_type = 'c',
                                                             class_count = 5,
                                                             scheme = 'quantile',
                                                             col = 'Vulnerability_Index_Percentile',
                                                             cmap = 'RdYlBu_r',
                                                             line_width = .01,
                                                             vmin = 0,
                                                             vmax=100
                                                            )

    # Define a function to be called when the "Update" button is clicked
    def on_update_button_click(click):
        print("Please wait for map to update")
        update_dataframe(None)
        update_map(None)
        print("Map update complete.")
        # Create a download link widget
        download_link = create_download_link(df)
        # Display the download link
        display(download_link)

    # Define Indicator Columns
    def define_columns_to_rank():  
        # columns_to_rank = [col for col in sdf.columns if '_PREPROCESSED' in col]
        columns_to_rank = ['STUNTING_SUM_PREPROCESSED',
                           'sum_Length_METERS_PREPROCESSED',
                           'anomaly_rate_PREPROCESSED',
                           'IPC_Average_PREPROCESSED',
                           'Affected_ADJ_PREPROCESSED',
                           'CropDMGHA_ADJ_PREPROCESSED',
                           'DahaloFlagActor1Cnt_PREPROCESSE',
                          'DahaloFlagActor2Cnt_PREPROCESSE',
                           'market_price_volatility_PREPROC',
                          ]
        return(columns_to_rank)

    def display_map(df):
        m1 = GIS().map(
           "Anosy",
            zoomlevel=7
        )
        m1.basemap = 'hybrid'
        m1.legend = True
        m1.basemap_switcher = True
        m1.layout = {
            'width' : '1000px',
            'height' : '600px'
    #         'width' : '1920px',
    #         'height' : '1080px'
        }

        df[['SHAPE','Vulnerability_Index_Percentile']].spatial.plot(map_widget = m1,
                                                             renderer_type = 'c',
                                                             class_count = 5,
                                                             scheme = 'quantile',
                                                             col = 'Vulnerability_Index_Percentile',
                                                             cmap = 'RdYlBu_r',
                                                             line_width = .01,
                                                             vmin = 0,
                                                             vmax=100
                                                            )
#         display(m1)
        return(m1)
    
    # Function to create a download link for DataFrame as CSV
    def create_download_link(df, filename='Updated_Indicator_Weights.csv'): 
        cols_to_download = ['OBJECTID', 'SOURCE_ID', 'Vulnerability_Index','Vulnerability_Index_Percentile'] + rank_cols + adj_weight_cols 
        csv_str = df[cols_to_download].to_csv(index=False)
        csv_bytes = csv_str.encode('utf-8')
        b64 = base64.b64encode(csv_bytes)
        payload = b64.decode()
        download_link = f'<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">Download CSV with updated indicator weights and weighted vulnerability index </a>'
        return widgets.HTML(download_link)
    
    # Load Dataframe with Initial Vulnerability Index (Default Weights = 0.1)
    sdf = load_sdf(itemID)
    df, rank_cols, adj_weight_cols = create_percentile_rank_index(sdf, define_columns_to_rank())
    m1 = display_map(df)
    
    # Create a widget title
    VBox_Title = widgets.HTML(value = "<h2 style='font-weight: bold; text-decoration: underline; '>Vulnerability Index Percentile Map</h2>")

    # Create an "Update" button
    update_button = widgets.Button(description="Click to update")

    # Create an output widget to display the updated DataFrame
    output_widget = widgets.Output()

    # Create a dictionary to store the input widgets
    input_widgets = {}

    # Create input widgets for each column with the column name as the description
    for column in adj_weight_cols:
    #     input_widgets[column] = widgets.IntText(value=df[column][0], description=column)
        input_widgets[column] = widgets.FloatSlider(
            value=df[column][0],
            min=0,
            max=1,
            step=0.1,
            description=column.rsplit('_', 2)[0],
            orientation='horizontal',
            layout=widgets.Layout(width='50%'),  # Allows the label to expand as needed
            style={'description_width': '10%'},  # Prevents line breaks in the label text
            continuous_update=False,  # Prevents real-time updates
        )
        input_widgets[column].title = column  # Set the tooltip as the column name

    # Attach the update_dataframe function to the change events of the input widgets
    for column in adj_weight_cols:
        input_widgets[column].observe(update_dataframe, 'value')


    # Attach the on_update_button_click function to the button's click event
    update_button.on_click(on_update_button_click)

    # Display the input widgets, the "Update" button, and the updated DataFrame
    input_widgets_list = [input_widgets[column] for column in adj_weight_cols]
    
    # Create a widget title
    weight_slider_title = widgets.HTML(value = "<h3 style='font-weight: bold;'>Interactive Weight Slider</h3>")
    
    # Group widgets in VBox
    VBox = widgets.VBox([VBox_Title, m1, update_button, weight_slider_title, *input_widgets_list, output_widget])
#     VBox.layout.display = 'flex'
#     VBox.layout.flew_flow = 'column'
#     VBox.layout.align_items = 'stretch'
    VBox.layout.width = '100%'
    VBox.layout.heigth = '300%'
    display(VBox)
    
    return(df)

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 7.63 µs


### Run Vulnerability Index Explorer: Instructions
    1) Use the slider to adjust the indicator weights.
    2) Click the button underneathe the map to update the map.
    3) Once the map updates, you will have the option to download an excel file with your updated weighted indicators and vulnerability index. 

In [2]:
%%time
df = Run_Vulnerability_Index_Explorer(itemID="40a219e06df749c0b792d6e8f03ebc3a")  # The itemID is associated with a feature layer on ArcGIS Online. 

CPU times: user 5 s, sys: 198 ms, total: 5.2 s
Wall time: 7.56 s
Please wait for map to update
Map update complete.


HTML(value='<a download="Updated_Indicator_Weights.csv" href="data:text/csv;base64,T0JKRUNUSUQsU09VUkNFX0lELFZ…